<a href="https://colab.research.google.com/github/ichekhovskikh/elibrary-pdf-downloader/blob/master/elibrary_pdf_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install requests
!pip install PyPDF2

import requests
import re
from PyPDF2 import PdfFileReader
from google.colab import drive

Указание логина и пароля для доступа к eLibrary. так же требуется указать строку поиска и количество страниц для просмотра

In [0]:
login = 'your_login'
password = 'your_password'
search = 'космические снимки'
page_count = 10

Монтирование директории для работы с файлами

In [0]:
drive.mount('/content/drive')

Инициализация сессии для работы с eLibrary

In [0]:
save_path = '/content/drive/data/{}/{}.txt'

headers = {'content-type': 'application/x-www-form-urlencoded','User-Agent': 'PostmanRuntime/7.19.0'}
params = {'login': login,'password': password}

auth_url = 'https://elibrary.ru/start_session.asp'
search_url = 'https://elibrary.ru/query_results.asp?where_abstract=on&type_article=on&queryboxid=&type_report=on&type_preprint=on&search_open=&rubrics_all=&changed=1&where_fulltext=on&where_keywords=on&type_patent=on&search_freetext=&search_fulltext=&where_name=on&where_references=on&itemboxid=&end_year=&titles_all=&where_affiliation=on&begin_year=&order=rev&type_conf=on&search_morph=on&authors_all=&type_disser=on&search_results=&type_book=on&ftext={}&issues=all&orderby=rank&'.format(search)
search_page_url = 'https://elibrary.ru/query_results.asp?pagenum={}'
download_pdf_url = 'https://elibrary.ru/full_text.asp?id={}'

session = requests.Session()
session.post(auth_url, data=params, headers=headers)
session.get(search_url, headers=headers)

Получение идентификаторов статей, доступных для скачивания в pdf

In [0]:
def available_pdf_ids(html_page):
   matches = re.findall(r"#pdf_\d+", html_page)
   for match in matches:
     yield match.replace("#pdf_", "")

Ковертирование pdf в text

In [0]:
def pdf2text(pdf_file):
    pdf = PyPDF2.PdfFileReader(pdf_file)
    text = []
    for page in pdf.pages:
      text.append(page.extractText())
    return text.replace("\n", ' ')

Скачивание статьи

In [0]:
def download_pdf(path, pdf_id):
  response = session.get(download_pdf_url.format(pdf_id), headers=headers)
  pdf_buffer = StringIO.StringIO().write(response.content)
  text = pdf2text(pdf_buffer)
  with open(path, 'w') as text_file:
    text_file.write(text)

In [0]:
for page_index in range(page_count):
  response = session.get(search_page_url.format(page_index), headers=headers)
  pdf_ids = available_pdf_ids(html_page)
  if not pdf_ids:
    break
  for pdf_id in pdf_ids:
    download_pdf(save_path.format(search, pdf_id), pdf_id)